#CUPED (Controlled-experiment Using Pre-Experiment Data)

### Материал для ноутбука взят из публикации на Хабр "Как улучшить ваши A/B-тесты: лайфхаки аналитиков Авито. Часть 2". Автор Дмитрий Лунин

In [ ]:
from collections import namedtuple
import scipy.stats as sps
import statsmodels.stats.api as sms
from tqdm.notebook import tqdm as tqdm_notebook
from collections import defaultdict
from statsmodels.stats.proportion import proportion_confint
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, palette='Set2')
ExperimentComparisonResults = namedtuple('ExperimentComparisonResults', 
                                        ['pvalue', 'effect', 'ci_length', 'left_bound', 'right_bound'])

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def absolute_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    var_mean_control  = np.var(control) / len(control)
    var_mean_test  = np.var(test) / len(test)
    
    difference_mean = mean_test - mean_control
    difference_mean_var = var_mean_control + var_mean_test
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([0.025, 0.975])
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

In [ ]:
def cuped_ttest(control, test, control_before, test_before):
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) /\
                (np.var(control_before) + np.var(test_before))

    control_cup = control - theta * control_before
    test_cup = test - theta * test_before
    return absolute_ttest(control_cup, test_cup)

In [ ]:
bad_cnt = 0

In [ ]:
N = 30000
for i in tqdm_notebook(range(N)):

    control_before = sps.expon(scale=1000).rvs(1000)
    control = control_before + sps.norm(loc=0, scale=100).rvs(1000)

    test_before = sps.expon(scale=1000).rvs(1000)
    test = test_before + sps.norm(loc=0, scale=100).rvs(1000)
    test *= 1.1

    _, _, _, left_bound, right_bound = cuped_ttest(control, test, control_before, test_before)
    
    if left_bound > 100 or right_bound < 100:
        bad_cnt += 1
        
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')

print(f"Реальный уровень значимости: {round(bad_cnt / N, 4)};"
      f" доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]")

  0%|          | 0/30000 [00:00<?, ?it/s]

Реальный уровень значимости: 0.0499; доверительный интервал: [0.0475, 0.0524]


In [ ]:
cuped_ci_lengths = []
ttest_ci_lengths = []
N = 30000
for i in tqdm_notebook(range(N)):

    control_before = sps.expon(scale=1000).rvs(1000)
    control = control_before + sps.norm(loc=0, scale=100).rvs(1000)

    test_before = sps.expon(scale=1000).rvs(1000)
    test = test_before + sps.norm(loc=0, scale=100).rvs(1000)
    test *= 1.1

    _, _, cuped_ci, _, _ = cuped_ttest(control, test, control_before, test_before)
    _, _, ttest_ci, _, _ = absolute_ttest(control, test)

    cuped_ci_lengths.append(cuped_ci)
    ttest_ci_lengths.append(ttest_ci)
    
coeff = np.mean(cuped_ci_lengths) / np.mean(ttest_ci_lengths)
print(f"Отношение ширины доверительных интервалов друг к другу: {round(coeff * 100, 3)}%")

  0%|          | 0/30000 [00:00<?, ?it/s]

Отношение ширины доверительных интервалов друг к другу: 11.016%
